In [1]:
import pandas as pd
import urllib.request
import urllib.parse
from bs4 import BeautifulSoup
import re

baseUrl = 'https://transcripts.foreverdreaming.org'
start = 8536

In [2]:
def getLinks(url):
    links = set()
    basepage = urllib.request.urlopen(url)
    soup = BeautifulSoup(basepage)
    for link in soup.find_all('h3'):
        if (re.search("([0-9]x[0-9])", link.text)):
            for part in link.findChildren('a'):
                links.add(part['href'][1:])
    return links

In [3]:
def getDialogue(baseUrl, predicate):
    url = baseUrl + predicate
    try:
        basepage = urllib.request.urlopen(url)
    except Exception as e:
        return [],[]
    soup = BeautifulSoup(basepage)
    speakerList = []
    lineList = []
    for line in soup.find_all("p"):
        if (len(line.attrs) == 0):
            if ':' in line.text:
                sepLine = line.text.split(':')
                if '(' in sepLine[0]:
                    speakerList.append(sepLine[0].split(' ')[0])
                else:
                    speakerList.append(sepLine[0])
                lineList.append(sepLine[1])
    return speakerList, lineList

In [4]:
allQueries = set()
base = 'https://transcripts.foreverdreaming.org/viewforum.php?f=159&start='
for i in range(0,151,25):
    allQueries.update(getLinks(base + str(i)))
allQueries.update(getLinks(base + str(158)))

In [6]:
speakerList = []
lineList = []
for query in allQueries:
    a,b = getDialogue(baseUrl, query)
    speakerList = speakerList + a
    lineList = lineList + b

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


In [12]:
speakerLine = []
sheldonLine = []
for i in range(len(speakerList)):
    if(speakerList[i] == 'Sheldon'):
        if(speakerList[i-1] != 'Sheldon' and speakerList[i-1] != 'Scene'):
            speakerLine.append(lineList[i-1])
            sheldonLine.append(lineList[i])

In [13]:
df = pd.DataFrame(list(zip(speakerLine, sheldonLine)), columns = ['speaker', 'response'])

In [14]:
df.to_csv('dataset.csv', index = False)